# Lab | APIS

In this lab, you will collect historical data about the Nobel Prize winners using [this free and non-authenticated API](https://www.nobelprize.org/organization/developer-zone-2/). According to the documentation available [here](https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1#/default/get_nobelPrizes). The base url is: "http://api.nobelprize.org/2.1/" followed by a string to specify what kind of information do you want to retrieve. The acceptable options are:

* nobelPrices
* nobelPrice/category/year
* laureates
* laureate/laureateID

# Getting the information using requests

Use the Python `requests`, and `json` libraries to obtain the information of ALL the Nobel Prizes. Make sure to verify that you get the proper status code (200).

The json outputs are simple plain text that need to be converted into the corresponding nested dictionary. Use the `.json()` method to cast the output into a Python dictionary.

Use the Pandas library to collect all the information into a Panda's DataFrame.

In [3]:
import requests
import json
import pandas as pd
from datetime import datetime

url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
        print("All good!")
        print("==============")
        print("\n")
        # Your code here

All good!




# Processing the output

Process the Pandas DataFrame in order to have only the following columns:

- category
- dateAwarded (as DateTime in "yyyy-mm-dd" format)
- prizeAmount
- prizeAmountAdjusted
- Number_of_laureates
- motivation
- laureate_ids (as a list)

In [39]:
url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
    prizes_list = response.json()['nobelPrizes'] # Convert the response to a JSON object
    # Your code here
    #print(prizes_list[0])

{'awardYear': '1901', 'category': {'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}, 'categoryFullName': {'en': 'The Nobel Prize in Chemistry', 'no': 'Nobelprisen i kjemi', 'se': 'Nobelpriset i kemi'}, 'dateAwarded': '1901-11-12', 'prizeAmount': 150782, 'prizeAmountAdjusted': 10531894, 'links': [{'rel': 'nobelPrize', 'href': 'https://api.nobelprize.org/2/nobelPrize/che/1901', 'action': 'GET', 'types': 'application/json'}], 'laureates': [{'id': '160', 'knownName': {'en': "Jacobus H. van 't Hoff"}, 'fullName': {'en': "Jacobus Henricus van 't Hoff"}, 'portion': '1', 'sortOrder': '1', 'motivation': {'en': 'in recognition of the extraordinary services he has rendered by the discovery of the laws of chemical dynamics and osmotic pressure in solutions', 'se': 'såsom ett erkännande av den utomordentliga förtjänst han inlagt genom upptäckten av lagarna för den kemiska dynamiken och för det osmotiska trycket i lösningar'}, 'links': [{'rel': 'laureate', 'href': 'https://api.nobelprize.org/2/laurea

In [33]:
# Initialize lists to hold the data
categories = []
dates_awarded = []
prize_amounts = []
prize_amount_adjusted = []
number_of_laureates = []
motivations = []
laureate_ids = []

In [57]:
# Loop through the prizes and extract the relevant data
# category:

for prize in prizes_list:
    # Category
    categories.append(prize['category']['en'])

display(categories[:5])

['Chemistry', 'Literature', 'Peace', 'Physics', 'Physiology or Medicine']

In [67]:
# dateAwarded (as DateTime in "yyyy-mm-dd" format)

for prize in prizes_list:
    # Date awarded
    dates_awarded.append(prize['awardYear'])

display(dates_awarded[:5])

['1901', '1901', '1901', '1901', '1901']

In [83]:
# prizeAmount:

for prize in prizes_list:
    # prizeAmount
    prize_amounts.append(prize['prizeAmount'])

display(prize_amounts[:5])

[150782, 150782, 150782, 150782, 150782]

In [63]:
# prizeAmountAdjusted:

for prize in prizes_list:
    # prizeAmountAdjusted
    prize_amount_adjusted.append(prize['prizeAmountAdjusted'])

display(prize_amount_adjusted[:5])

[10531894, 10531894, 10531894, 10531894, 10531894]

In [93]:
# Number_of_laureates:

for prize in prizes_list:
    # Number of laureates
    number_of_laureates.append(len(prize['laureates']) if 'laureates' in prize else 0)

display(number_of_laureates[:5])

[1, 1, 2, 1, 1]

In [101]:
# motivation:

for prize in prizes_list:
    # Motivation
    #motivations.append(prize['laureates']['motivation']['en'])
    motivations.append(", ".join([laureate['motivation']['en'] for laureate in prize['laureates']]) if 'laureates' in prize else "")

display(motivations[:5])

['in recognition of the extraordinary services he has rendered by the discovery of the laws of chemical dynamics and osmotic pressure in solutions',
 'in special recognition of his poetic composition, which gives evidence of lofty idealism, artistic perfection and a rare combination of the qualities of both heart and intellect',
 'for his humanitarian efforts to help wounded soldiers and create international understanding, for his lifelong work for international peace conferences, diplomacy and arbitration',
 'in recognition of the extraordinary services he has rendered by the discovery of the remarkable rays subsequently named after him',
 'for his work on serum therapy, especially its application against diphtheria, by which he has opened a new road in the domain of medical science and thereby placed in the hands of the physician a victorious weapon against illness and deaths']

In [103]:
# laureate_ids (as a list):

for prize in prizes_list:
    # Laureate IDs
    #laureate_ids.append(prize['laureates']['id'])
    laureate_ids.append([laureate['id'] for laureate in prize['laureates']] if 'laureates' in prize else [])


display(laureate_ids[:5])

[['160'], ['569'], ['462', '463'], ['1'], ['293']]

# Getting a Pandas DataFrame with the details of awarded authors/institutions

If you dive deeper and use the API to retrieve the details of some laureate_ids, you will notice that not allways the Nobel Prize was awarded to individuals. In some cases, the awards were given to institutions.

Get the unique ids from the previous datasets and prepare the following functions:

- get_name(laureate) ( it should return the english name 'fullName' of the individual or 'orgName' of the institution )

- get_gender(laureate) ( it should return the gender or 'Unknown' for individuals, and 'None' for institutions )

- get_birthdate(laureate) ( it should return the birthdate when it's avaialble or 'Unknown' otherwise )

- get_age(laureate) ( it should return the age of the awarded individual or 'Unknown' when it's not avaialble or for institutions )

- get_city(laureate) ( it should return the english name of the city when it's available or 'Unknown' otherwise )

- get_country(laureate) ( it should return the english name of the country when it's available or 'Unknown' otherwise )

- get_continent(laureate) ( it should return the english name of the continent when it's available or 'Unknown' otherwise )

- get_latitude(laureate) ( it should return the city's latitude when it's available or 'Unknown' otherwise )

- get_longitude(laureate) ( it should return the city's longitude
 when it's available or 'Unknown' otherwise )

Create the following dictionaries:

```python
laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}                        

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}
```

For each unique `laureate_id` of the previous DataFrame make an API call to get the details of the awarded individual/intitution and iterate of the previous dictionaries keys in order to add the corresponding information of each `laureate_id` in the empty lists of `laureates_dict`.

Finally, create a Pandas DataFrame named `laureates_df` using the `laureates_dict`.

In [ ]:
import time
from tqdm import tqdm


ids = [int(item) for l in df['laureate_ids'].values for item in l]
unique_ids = set(ids)

def get_name(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here


def get_gender(laureate):

    # Person
    # Your code here

    # Organizrtion
    # Your code here

def get_birthdate(laureate):

    birth_date = "Unknown"

    # Person
    # Your code here

    # Organization
    # Yuor code here

    return birth_date

def get_age(laureate):

    birth_date = "Unknown"
    award_date = "Unknown"

    # Person
    # Your code here

    # Organization
    # Yuor code here

    # Award date
    # Your code here

def get_city(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here

def get_country(laureate):

    # Person
    # Your code here

    # Origanization
    # Your code here


def get_continent(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here

def get_latitude(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here

def get_longitude(laureate):

    # Person
    # Your code here

    # Organization
    # Yuor code here


laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}

for index, id in enumerate(tqdm(unique_ids)):

    url = "https://api.nobelprize.org/2/laureate/" + str(id)
    response = requests.get(url)

    if response.status_code == 200:

        laureate = response.json()

        # Your code here

laureates_df = pd.DataFrame(laureates_dict)

laureates_df



100%|██████████| 992/992 [10:16<00:00,  1.61it/s]


,ID,Name,Gender,Birth_date,Age,City,Country,Continent,Latitude,Longitude
0,1,Wilhelm Conrad Röntgen,male,1845-03-27 00:00:00,None,Lennep,Germany,Europe,51.178742,7.189696
1,2,Hendrik Antoon Lorentz,male,1853-07-18 00:00:00,None,Arnhem,the Netherlands,Europe,51.984257,5.910857
2,3,Pieter Zeeman,male,1865-05-25 00:00:00,None,Zonnemaire,the Netherlands,Europe,51.713056,3.951111
3,4,Antoine Henri Becquerel,male,1852-12-15 00:00:00,None,Paris,France,Europe,48.860093,2.355954
4,5,Pierre Curie,male,1859-05-15 00:00:00,None,Paris,France,Europe,48.860093,2.355954
...,...,...,...,...,...,...,...,...,...,...
987,1030,Louis E. Brus,male,Unknown,None,"Cleveland, OH",USA,North America,41.496386,-81.710675
988,1031,Aleksey Yekimov,male,Unknown,None,Leningrad,Russia,Europe,59.956651,30.333547
989,1032,Jon Fosse,male,1959-09-29 00:00:00,None,Haugesund,Norway,Europe,59.410150,5.275511
990,1033,Narges Mohammadi,female,1972-04-21 00:00:00,None,Zanjan,Iran,Asia,36.666667,48.483333


# Country ranking

Get a ranking countries by the number of times that they had been awarded in any category.

In [ ]:
# Your code here

,ID
Country,
USA,296
United Kingdom,91
Germany,84
France,63
Russia,30
...,...
Greece,1
Ghana,1
Faroe Islands (Denmark),1
